# Ridge Regression from Scratch in Python

In the Jupyter Notebook, a Ridge regression machine learning algorithm using Python programming is implemented from scratch for educational purposes. The Ridge regression model makes predictions about a person's salary. If an employee has more experience and is older, the higher output salary is predicted from the model. The less experience and the younger an employee is the lower the salary output prediction from the model. The ScikitLearn machine learning library with NumPy, Matplotlib, Pandas and Scipy has been used to pre-process the data and evaluate performance metrics from the Ridge regression model. The ScikitLearn Ridge regression algorithm has been implemented at the end of the Jupyter Notebook to compare the evaluation test results against the Python Ridge regression algorithm.

The Ridge regression algorithm adds an L2 regularisation penalty to the model to reduce overfitting. L2 regularisation is where the squared sum of the weights is added to the result of the least squares cost function during model training.

Although this is a very small dataset for a statistical model to make predictions on, it provides an introduction to machine learning algorithms and automatic differentiation. A larger dataset may return missing values (NaN) as output predictions over other Ridge regression algorithms written in software using well-optimised machine learning libraries such as ScikitLearn, Pytorch or TensorFlow. Larger datasets require optimised data loading software that is not added to the Ridge regression model object to reduce code complexity for educational purposes.


# 1. Ridge Regression Object

In [1]:
import numpy as np

# Regression class object to prcess continious data
class Regression:

    # Regression class constructor
    def __init__(self, regularization, lr, n_iters):
        self.n_samples = None   # Dataset samples
        self.features = None    # Features from the dataset
        self.weights = None     # weight parameters
        self.bias = None        # bias parameter
        self.regularization = regularization  # rgularisation
        self.lr = lr            # learning rate
        self.n_iters = n_iters      # iterations

    # Regression class cost function to calculate the loss of the model to determine the performance
    def __calculate_cost(self, y, y_pred):

        # Calculates the cost of the regression model during training and returns the cost value
        return (1 / (2 * self.n_samples)) * np.sum(np.square(y_pred - y)) + self.regularization(self.weights)

    # Python fit method to fit the regression model during training
    def fit(self, X, y):

        # Retrieves the y_train target values dataset length in preperation for reshaping the dataset to 1 dimension
        dim_1 = len(y)

        # Reshapes the y_train target values for the Ridge Regression algorithm and converts it to a NumPy array
        y = y.values.reshape(dim_1, 1)

        # Initialises the wieght parameters before training
        X = np.insert(X, 0, 1, axis=1)

        # stores the number of samples and features from the dataset
        self.n_samples, self.features = X.shape

        # Updates the weight parameters after initilization
        self.weights = np.zeros((self.features, 1))

        # Training loop for the regression model
        for e in range(1, self.n_iters + 1):

            # Stores the training predictions made by the regression model
            y_pred = np.dot(X, self.weights)

            # Calculats the cost of the regression model during training
            cost = self.__calculate_cost(y, y_pred)

            # Partial differentiation for the weigh parameters
            dw = (1 / self.n_samples) * np.dot(X.T, (y_pred - y)) + self.regularization.derivation(self.weights)

            # Updates the weight parameter after each training iteration
            self.weights = self.weights - self.lr * dw

            # Prints the cost of hte regression model after every 100 iterations
            if e % 100 == 0:
                # Print statement to print the cost to the screen with an F string
                print(f"The Cost in iteration {e}:\t {cost}")

        # Prints out training complete to the screen
        print("Training Complete")

    # Python predict method to make predictions on the trained regression model at test time
    def predict(self, X_test):

        # Reshapes the test dataset X_test for the regression model
        X_test = np.insert(X_test, 0, 1, axis=1)

        # Stores the predictions made from the regression model at test time
        y_pred = np.dot(X_test, self.weights)

        # Returns the predictions made from the regression model at test time
        return y_pred

In [2]:
# RidgePenalty class object
class RidgePenalty:

    # RidgePenalty Class constructor
    def __init__(self, l):
        self.l = l  # lambda regularization parameter

    # Calculats the regularization lambda parameter's non-absolute value
    def __call__(self, w):
        # Returns the regularization lambda parameter's absolute value
        return self.l * np.sum(np.square(w))

    # Calculates the derivation of the Ridge regression model
    def derivation(self, w):
        # Returns the derivation of the Ridge regression model
        return self.l * 2 * w

In [3]:
# Ridge regression Python object that inherits the regression class object
class Ridge(Regression):

    # Ridge regression class constructor
    def __init__(self, l, lr, n_iters):
        # Stores the regularization parameter lambda calculated by the RidgePenalty class object
        self.regularization = RidgePenalty(l)

        # Calls the Regression class object to input the lambda regularisation poramter,
        # learning rate and number of iterations (n_iters)
        super().__init__(self.regularization, lr, n_iters)

# 2. Load the Dataset

Dataset Available from Kaggle:

https://www.kaggle.com/datasets/hussainnasirkhan/multiple-linear-regression-dataset

In [4]:
# Imports the Pandas library to use in the Jupyter Notebook
import pandas as pd

# Load the data to the Jupyter Notebook
df = pd.read_csv('./dataset/multiple_linear_regression_dataset.csv')

df

,age,experience,income
0,25,1,30450
1,30,3,35670
2,47,2,31580
3,32,5,40130
4,43,10,47830
5,51,7,41630
6,28,5,41340
7,33,4,37650
8,37,5,40250
9,39,8,45150


In [5]:
# Seperate the target values from the dataset using the Pandas Pop method
y = df.pop('income')

y

0     30450
1     35670
2     31580
3     40130
4     47830
5     41630
6     41340
7     37650
8     40250
9     45150
10    27840
11    46110
12    36720
13    34800
14    51300
15    38900
16    63600
17    30870
18    44190
19    48700
Name: income, dtype: int64

In [6]:
# Displays the dataset features X to the screen
X = df

X

,age,experience
0,25,1
1,30,3
2,47,2
3,32,5
4,43,10
5,51,7
6,28,5
7,33,4
8,37,5
9,39,8


# 3. Normalise the Data

In [7]:
# Convert column names to a list
list_numerical = X.columns.tolist()

list_numerical

['age', 'experience']

In [8]:
# Python method to normalise the features X of the dataset 
def get_scale(list_numerical, X):
    
    # Import the StandardScaler object from ScikitLearn to normalise the data
    from sklearn.preprocessing import StandardScaler
    
    # Creates the StandardScaler object from ScikitLearn
    scaler = StandardScaler().fit(X[list_numerical]) 

    # Fits the normalisation to the dataset features X
    X[list_numerical] = scaler.transform(X[list_numerical])

    # Returns the normalised dataset features X
    return X

In [9]:
# Calls the get_scale Python method to normalise the dataset features X
X = get_scale(list_numerical, X)

X

,age,experience
0,-1.498903,-1.293548
1,-0.987332,-0.796030
2,0.752009,-1.044789
3,-0.782703,-0.298511
4,0.342752,0.945285
5,1.161266,0.199007
6,-1.191960,-0.298511
7,-0.680389,-0.547270
8,-0.271133,-0.298511
9,-0.066504,0.447767


# 4. Train Test Split

In [10]:
# Imports the train test split method from ScikitLearn
from sklearn.model_selection import train_test_split

# Carry out a train, test split from ScikitLearn
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Prints out the X_train features to the screen
X_train

,age,experience
8,-0.271133,-0.298511
5,1.161266,0.199007
11,0.752009,0.696526
3,-0.782703,-0.298511
18,0.445067,0.696526
16,1.877465,2.686600
13,1.161266,-0.547270
2,0.752009,-1.044789
9,-0.066504,0.447767
19,-0.271133,0.945285


In [11]:
# Prints out the y_train targets to the screen
y_train

8     40250
5     41630
11    46110
3     40130
18    44190
16    63600
13    34800
2     31580
9     45150
19    48700
4     47830
12    36720
7     37650
10    27840
14    51300
6     41340
Name: income, dtype: int64

In [12]:
# Prints out the X_test features to the screen
X_test

,age,experience
0,-1.498903,-1.293548
17,-1.703531,-1.293548
15,0.138124,-0.049752
1,-0.987332,-0.796030


In [13]:
# Prints out the y_test targets to the screen
y_test

0     30450
17    30870
15    38900
1     35670
Name: income, dtype: int64

# 5. Train the Ridge Regression Model

In [14]:
# Stores the input the parameters to the Ridge regression algorithm
parameters = {
    "l" : 0.1,
    "lr" : 0.1,
    "n_iters" : 300
}

# Creating the Ridge Regression class object
model = Ridge(**parameters)

# Fits the data to the Ridge regression algorithm during training
model.fit(X_train, y_train)

The Cost in iteration 100:	 144923076.95517254
The Cost in iteration 200:	 144923051.19726372
The Cost in iteration 300:	 144923051.19723445
Training Complete


# 6. Make Predictions on the Test Dataset

In [15]:
# Makes predictions on the test dataset X_test
y_pred = model.predict(X_test)

y_pred

array([[21872.08647974],
       [21596.15679342],
       [33521.876166  ],
       [26338.85157382]])

In [16]:
# Flattens the y_test 2D array to 1D and stores it in the y variable
y_pred = y_pred.flatten()

y_pred

array([21872.08647974, 21596.15679342, 33521.876166  , 26338.85157382])

In [17]:
# Prints out the y_test variable to the screen
y_test

0     30450
17    30870
15    38900
1     35670
Name: income, dtype: int64

In [18]:
# Flattens the y_test 2D array to 1D and stores it in the y variable
y = y_test.values.flatten()

y

array([30450, 30870, 38900, 35670])

# 7. PLCC, SRCC and KRCC Correlation Performance Metrics Method

In [19]:
# Python method to calculate correlation coefficient metrics
def correlation(X, y):
    
    # Import Scipy libraray
    import scipy.stats
    
    # Calculate Pearson's Linear Correlation Coefficient with Scipy
    PLCC = scipy.stats.pearsonr(X, y)[0]    # Pearson's r

    # Calculate Spearman Rank Correlation Coefficient (SRCC) with Scipy
    SRCC = scipy.stats.spearmanr(X, y)[0]   # Spearman's rho

    # Calculate Kendalls Rank Correlation Coefficient (KRCC) with Scipy
    KRCC = scipy.stats.kendalltau(X, y)[0]  # Kendall's tau
    
    # Prints out the correlation performance metric results to the screen
    print("PLCC: ", PLCC)
    print("SRCC: ", SRCC)
    print("KRCC: ", KRCC)

# 8. Evaluate PLCC, SRCC and KRCC Performance Metrics

In [20]:
# Calls the correlation coefficient Python method to calculate PLCC, SRCC, and KRCC
# with SciPy
correlation(y_pred, y)

PLCC:  0.973354828855769
SRCC:  0.7999999999999999
KRCC:  0.6666666666666669


# Root Mean Square Error (RMSE)

In [21]:
# Imports the mean_squared_error method from ScikitLearn
from sklearn.metrics import mean_squared_error

# Calls the Mean Squared Error method from ScikitLearn
rmse = mean_squared_error(y_pred, y)

# Prints out the RMSE value to the screen
np.sqrt(rmse)

8300.290885063903

# R2 Score

In [22]:
# Imports the r2_score method from ScikitLearn
from sklearn.metrics import r2_score

# Calls the R2 Score method from ScikitLearn
r2 = r2_score(y_pred, y)

# Prints out the R2 Score value to the screen
r2

-1.9627604410004014

# 9. Test on the ScikitLearn Ridge Regression Model

Ridge Regression ScikitLearn Documentation Link:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

## Train the Model

In [23]:
# Imports the Ridge Regression model from ScikitLearn
from sklearn.linear_model import Ridge

# Trains the ScikitLearn Ridge Regression model
model = Ridge(alpha=0.1).fit(X_train, y_train)

# 10. Make Predictions

In [24]:
# Makes predictions on the Linear Regression model
y_pred = model.predict(X_test)

y_pred

array([31127.35941867, 31320.11442125, 40265.20005034, 34917.42417314])

In [25]:
# Flattens the 2D pred array to 1D from the model and stores it in the flat Python variable
flat = y_pred.flatten()

flat

array([31127.35941867, 31320.11442125, 40265.20005034, 34917.42417314])

In [26]:
# Prints out the y_test variable to the screen
y_test

0     30450
17    30870
15    38900
1     35670
Name: income, dtype: int64

# 11. Test the ScikitLearn Ridge Regression Model

In [27]:
# Calls the correlation coefficient Python method to calculate PLCC, SRCC, and KRCC
# with SciPy
correlation(flat, y)

PLCC:  0.9790186714117106
SRCC:  1.0
KRCC:  1.0


# Root Mean Square Error (RMSE)

In [28]:
# Imports the mean_squared_error method from ScikitLearn
from sklearn.metrics import mean_squared_error

# Calls the Mean Squared Error method from ScikitLearn
rmse = mean_squared_error(flat, y)

# Prints out the RMSE value to the screen
np.sqrt(rmse)

879.1416732939614

# R2 Score

In [29]:
# Imports the r2_score method from ScikitLearn
from sklearn.metrics import r2_score

# Calls the R2 Score method from ScikitLearn
r2 = r2_score(flat, y)

# Prints out the R2 Score value to the screen
r2

0.9436504182132818

# 12. Display Jupyter Notebook Python Variables

In [30]:
# Prints out all the Python variables from the Jupyter Notebook
%whos

Variable             Type         Data/Info
-------------------------------------------
Regression           type         <class '__main__.Regression'>
Ridge                ABCMeta      <class 'sklearn.linear_model._ridge.Ridge'>
RidgePenalty         type         <class '__main__.RidgePenalty'>
X                    DataFrame             age  experience\<...>n19 -0.271133    0.945285
X_test               DataFrame             age  experience\<...>n1  -0.987332   -0.796030
X_train              DataFrame             age  experience\<...>n6  -1.191960   -0.298511
correlation          function     <function correlation at 0x7d7ed0568900>
df                   DataFrame             age  experience\<...>n19 -0.271133    0.945285
flat                 ndarray      4: 4 elems, type `float64`, 32 bytes
get_scale            function     <function get_scale at 0x7d7ed4d4d580>
list_numerical       list         n=2
mean_squared_error   function     <function mean_squared_error at 0x7d7ed04edc60>
model